### Crypto News 2013-2017

##### data source for 2013-2018... https://www.kaggle.com/datasets/kashnitsky/news-about-major-cryptocurrencies-20132018-40k

In [46]:
# Substitute with your actual info
storage_account = "ds562team9datalake"
container_name = "bronze"
file_path = "Historical_news_on_crypto/crypto_news_2013-2017.csv"
storage_key = "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw==" 

spark.conf.set(f"fs.azure.account.key.{storage_account}.blob.core.windows.net", storage_key)

url = f"wasbs://{container_name}@{storage_account}.blob.core.windows.net/{file_path}"

df_crypto_news_2013_2017 = spark.read.option("header", True).csv(url)

In [47]:
storage_account = "ds562team9datalake"
container = "bronze"
file_path = "Historical_news_on_crypto/crypto_news_2013-2017.csv"

wasbs_path = f"wasbs://{container}@{storage_account}.blob.core.windows.net/{file_path}"

df_crypto_news_2013_2017 = spark.read.option("header", True) \
               .option("multiline", True) \
               .option("escape", "\"") \
               .option("quote", "\"") \
               .option("inferSchema", False) \
               .csv(wasbs_path)

df_crypto_news_2013_2017.printSchema()
df_crypto_news_2013_2017.select("title", "year", "source").show(10, truncate=True)


In [48]:
from pyspark.sql.functions import col, trim

missing_or_blank_count = df_crypto_news_2013_2017.filter(
    (col("title").isNull() | (trim(col("title")) == "")) |
    (col("text").isNull() | (trim(col("text")) == "")) |
    (col("year").isNull() | (trim(col("year")) == ""))
).count()
print("raw data row count:", df_crypto_news_2013_2017.count())
print(f"Rows with missing or blank 'title', 'text', or 'year': {missing_or_blank_count}")


In [49]:
from pyspark.sql.functions import length

df_clean_crypto_news_2013_2017 = df_crypto_news_2013_2017.filter(
    (col("title").isNotNull()) & (trim(col("title")) != "") &
    (col("text").isNotNull()) & (trim(col("text")) != "") &
    (col("year").isNotNull()) & (trim(col("year")) != "")
)

print("Cleaned row count:", df_clean_crypto_news_2013_2017.count())
print("Cleaned 2013-2017 row count:", df_clean_crypto_news_2013_2017.describe())

### Crypto News 2018

In [50]:
file_path_2018 = "Historical_news_on_crypto/crypto_news_2018.csv"
wasbs_path_2018 = f"wasbs://bronze@ds562team9datalake.blob.core.windows.net/{file_path_2018}"

df_2018 = spark.read.option("header", True).option("multiline", True).option("escape", "\"").option("quote", "\"").csv(wasbs_path_2018)

df_2018.printSchema()
df_2018.select("title", "year", "source").show(10, truncate=True)

In [51]:
missing_2018 = df_2018.filter(
    (col("title").isNull() | (trim(col("title")) == "")) |
    (col("text").isNull() | (trim(col("text")) == "")) |
    (col("year").isNull() | (trim(col("year")) == ""))
).count()

print(f"Rows with missing or blank 'title', 'text', or 'year' in 2018 dataset: {missing_2018}")

df_2018_clean = df_2018.filter(
    (col("title").isNotNull()) & (trim(col("title")) != "") &
    (col("text").isNotNull()) & (trim(col("text")) != "") &
    (col("year").isNotNull()) & (trim(col("year")) != "")
)

print("Original 2018 row count:", df_2018_clean.count())
print("Cleaned 2018 row count:", df_2018_clean.describe())

### Combining 2013-2017 with 2018

In [52]:
df_2013_2018_clean = df_clean_crypto_news_2013_2017.unionByName(df_2018_clean)

print("Row count - 2013–2017:", df_clean_crypto_news_2013_2017.count())
print("Row count - 2018:", df_2018_clean.count())
print("Row count - Combined (2013–2018):", df_2013_2018_clean.count())

print("Combined (2013–2018):", df_2013_2018_clean.describe())
print("Columns:", df_2013_2018_clean.columns)

### Combined Data 2013-2018

In [53]:
from pyspark.sql.functions import col, sum, when

null_counts = df_2013_2018_clean.select([
    sum(when(col(c).isNull() | (col(c) == ""), 1).otherwise(0)).alias(c)
    for c in df_2013_2018_clean.columns
])

null_counts.show()

In [54]:
df_final_trimmed = df_2013_2018_clean

In [55]:
from pyspark.sql.functions import col, lower

df_final_trimmed = df_final_trimmed.filter(
    lower(col("title")).contains("bitcoin") | 
    lower(col("title")).contains("btc") | 
    lower(col("text")).contains("bitcoin") | 
    lower(col("text")).contains("btc")
)

In [58]:
print("Columns:", df_final_trimmed.count())

In [59]:
spark.conf.set(
    "fs.azure.account.key.ds562team9datalake.blob.core.windows.net",
    "KXg2Djg7uRevBSpPNIVnKw/N6HpqBh+kJwDX07wkywbpU2joMZdTIBOXk30EoMMxH2d8wwb+9j0g+AStO60IWw=="
)

In [61]:
df_final_trimmed.coalesce(1).write.mode("overwrite").parquet(
    "wasbs://bronze@ds562team9datalake.blob.core.windows.net/Historical_news_on_crypto/cleaned_news_data"
)